# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [1]:
import json

# открываем файл и загружаем его содержимое
with open('data/contributors_sample.json') as f:
    contributors = json.load(f)

# выводим информацию о первых трех пользователях
for contributor in contributors[:3]:
    print(contributor)
    print()

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}

{'username': 'vickitaylor', 'name': 'Cheryl Lewis', 'sex': 'F', 'address': '66992 Welch Brooks\nMarshallshire, ID 56004', 'mail': 'bhudson@gmail.com', 'jobs': ['Music therapist', 'Volunteer coordinator', 'Designer, interior/spatial'], 'id': 91970}

{'username': 'sheilaadams', 'name': 'Julia Allen', 'sex': 'F', 'address': 'Unit 1632 Box 2971\nDPO AE 23297', 'mail': 'darren44@yahoo.com', 'jobs': ['Management consultant', 'Engineer, structural', 'Lecturer, higher education', 'Theatre manager', 'Designer, textile'], 'id': 1848091}



1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [2]:
# Создаем пустое множество для хранения уникальных доменов
domains = set()

# Проходимся по каждому человеку и добавляем его домен в множество
for person in contributors:
    email = person['mail']
    domain = email.split('@')[1]
    domains.add(domain)

# Выводим уникальные домены на экран
print(domains)

{'yahoo.com', 'hotmail.com', 'gmail.com'}


1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [3]:
# функция для поиска пользователя по username
def find_user(username):
    for user in contributors:
        if user['username'] == username:
            return user
    
    # если пользователь не найден, возбуждаем исключение ValueError
    raise ValueError(f"User with username '{username}' not found")

# пример использования функции
try:
    user = find_user('uhebert')
    print(user)
except ValueError as e:
    print(e)

{'username': 'uhebert', 'name': 'Lindsey Nguyen', 'sex': 'F', 'address': '01261 Cameron Spring\nTaylorfurt, AK 97791', 'mail': 'jsalazar@gmail.com', 'jobs': ['Energy engineer', 'Engineer, site', 'Environmental health practitioner', 'Biomedical scientist', 'Jewellery designer'], 'id': 35193}


1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [14]:
male_count = 0
female_count = 0
for item in contributors:
    if item['sex'] == 'M':
        male_count += 1
    elif item['sex'] == 'F':
        female_count += 1

# выводим результаты
print(f'Количество мужчин: {male_count}')
print(f'Количество женщин: {female_count}')

Количество мужчин: 2064
Количество женщин: 2136


1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [26]:
import pandas as pd
import json

# загрузка данных из файла
with open('data/contributors_sample.json') as f:
    data = json.load(f)

# выбор необходимых столбцов
data = [{'id': c['id'], 'username': c['username'], 'sex': c['sex']} for c in data]

# создание DataFrame
contributors = pd.DataFrame(data)

#Демонастрация работы:
contributors.head(10)

,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
5,64918,james67,F
6,113941,woodmarissa,M
7,398160,sampsontammy,M
8,35635,jonathan18,M
9,718054,michael53,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [41]:
recipes = pd.read_csv('data/recipes_sample.csv')

# объединение таблиц
recipes_contributors = recipes.merge(contributors, how='left', left_on='contributor_id', right_on='id')

# подсчет количества строк без информации о человеке
null_contributors = recipes_contributors['username'].isnull().sum()

print(f'Количество строк без информации о человеке: {null_contributors}')



Количество строк без информации о человеке: 15059


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [77]:
import json

# загрузка данных из файла
with open('data/contributors_sample.json') as f:
    data = json.load(f)

# создание словаря
positions = {}
for contributor in data:
    for job in contributor['jobs']:
        if job in positions:
            positions[job].append(contributor['username'])
        else:
            positions[job] = [contributor['username']]

# вывод результата, для получения результата необходимо раскомитеть, но будет много значений. 
#print(positions)

#Более "мягкая" демонстрация, работы, вывод первых 3 позиций. 
print(list(positions.items())[:3])

[('Energy engineer', ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones']), ('Engineer, site', ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria']), ('Environmental health practitioner', ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'j

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [66]:
import pickle
import json

# сохранение в pickle
with open('data/job_people.pickle', 'wb') as f:
    pickle.dump(positions, f)

# сохранение в JSON
with open('data/job_people.json', 'w') as f:
    json.dump(positions, f, indent=4)

In [64]:
import os

print(f"Размер файла job_people.pickle: {os.path.getsize('data/job_people.pickle')} байт")
print(f"Размер файла job_people.json: {os.path.getsize('data/job_people.json')} байт")

Размер файла job_people.pickle: 132272 байт
Размер файла job_people.json: 407711 байт


2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [71]:
with open('data/job_people.pickle', 'rb') as f:
    job_people = pickle.load(f)
    # вывод результата, для получения результата необходимо раскомитеть, но будет много значений. 
    #print(job_people)

#Более "мягкая" демонстрация, работы, вывод первых 3 позиций. 
print(list(job_people.items())[:3])

[('Energy engineer', ['uhebert', 'annmoore', 'garysilva', 'martinezashley', 'sextonsheila', 'pjames', 'smithjonathan', 'wardjames', 'cwheeler', 'ucarlson', 'robert71', 'johnsontheresa', 'amanda41', 'stacey47', 'timothynelson', 'timothynelson', 'rogersmichael', 'melissa94', 'wmcdaniel', 'charles74', 'smithjennifer', 'clintonjones']), ('Engineer, site', ['uhebert', 'nancy12', 'andrea03', 'catherineross', 'wesley32', 'natalieross', 'rossdoris', 'christophersmith', 'dbooker', 'ericarobertson', 'trantricia', 'tpugh', 'jasonvelez', 'samantha36', 'brandidaniels', 'tenglish', 'reyesbrett', 'austin18', 'vjohnson', 'zmejia', 'daniel04', 'cynthia20', 'morgan15', 'avaldez', 'jessica92', 'laurieholloway', 'baileyvictoria']), ('Environmental health practitioner', ['uhebert', 'jonathanchristian', 'xjohnson', 'dsmith', 'james01', 'nancytaylor', 'ztaylor', 'andrewwoods', 'susan54', 'fmaldonado', 'james74', 'bakerjacob', 'stephanie81', 'whitejoseph', 'qolson', 'hknox', 'gonzalesdaniel', 'tranronald', 'j

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [76]:
import xml.etree.ElementTree as ET
import json

# загрузка данных из файла
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()

#Создаем пустой словарь
steps_dict = {}

#Перебираем элементы внутри XML-файла, по тегу recipe
for recipe in root.iter('recipe'):
    #Забираем значение текста внутри тега id, текущего recipe
    recipe_id = recipe.find('id').text
    
    #Пустой список, для хранения полученых степов
    steps = []
    
    #Перебераем элементы тега step, который находится внутри steps
    for step in recipe.findall('steps/step'):
        #добавляем к нашему пустому списку, элемент тега step
        steps.append(step.text)
    #Формеруем словарь, делаем связку recipe_id и шагов.
    steps_dict[recipe_id] = steps

#Сохраняемся.
with open('data/steps_sample.json', 'w') as f:
    json.dump(steps_dict, f)

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [82]:
steps_count_dict = {}

for recipe in root.iter('recipe'):
    recipe_id = recipe.find('id').text
    steps_count = len(recipe.find('steps').findall('step'))
    if steps_count not in steps_count_dict:
        steps_count_dict[steps_count] = [recipe_id]
    else:
        steps_count_dict[steps_count].append(recipe_id)

#print(steps_count_dict)

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [85]:
recipes_with_time_info = []
for recipe in root.iter('recipe'):
    for step in recipe.findall('steps/step'):
        if 'has_minutes' in step.attrib or 'has_hours' in step.attrib:
            recipe_id = recipe.find('id').text
            if recipe_id not in recipes_with_time_info:
                recipes_with_time_info.append(recipe_id)

#print(recipes_with_time_info)

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [90]:
# Загрузка данных из CSV файла в датафрейм
recipes = pd.read_csv('data/recipes_sample.csv')

# Получение списка ID рецептов без информации о количестве шагов
missing_n_steps = recipes.loc[recipes['n_steps'].isna(), 'id'].tolist()

# Чтение данных из XML файла и заполнение пропусков в столбце n_steps
tree = ET.parse('data/steps_sample.xml')
root = tree.getroot()
for recipe in root.iter('recipe'):
    recipe_id = int(recipe.find('id').text)
    if recipe_id in missing_n_steps:
        steps = recipe.find('steps')
        n_steps = len(steps.findall('step'))
        recipes.loc[recipes['id'] == recipe_id, 'n_steps'] = n_steps

# Проверка результатов
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11.0,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3.0,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5.0,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,7.0,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [91]:
recipes[recipes.n_steps.isna()]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients


In [93]:
recipes.n_steps = recipes.n_steps.apply(int)
recipes.to_csv('./data/recipes_sample_with_filled_nsteps.csv')
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,7,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4,i think a fondue is a very romantic casual din...,NaN
